In [1]:
import numpy as np
np.random.seed(10)

import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve

# Import the required libraries.
import os
import cv2
#import pafy
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt

#from moviepy.editor import *
%matplotlib inline

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
tf.keras.backend.clear_session()
seed_constant = 27
np.random.seed(seed_constant)
tf.random.set_seed(seed_constant)
random.seed(seed_constant)
tf.keras.utils.set_random_seed(seed_constant)

In [3]:
# Create a Matplotlib figure and specify the size of the figure.
plt.figure(figsize = (20, 20))

# Get the names of all classes/categories in UCF50.
TRAIN_DIR = r'train'
VAL_DIR = r'val'
TEST_DIR = r'test'
all_classes_names = sorted(os.listdir(TRAIN_DIR))
print(all_classes_names)


['10outfinal', '1outfinal', '2outfinal', '3outfinal', '4outfinal', '5outfinal', '6outfinal', '7outfinal', '8outfinal', '9outfinal']


<Figure size 2000x2000 with 0 Axes>

In [4]:
# Specify the height and width to which each video frame will be resized in our dataset.
IMAGE_HEIGHT , IMAGE_WIDTH = 80, 80

# Specify the directory containing the UCF50 dataset. 
#DATASET_DIR = "/newdatasetMediapipe/kata/"

# Specify the list containing the names of the classes used for training. Feel free to choose any set of classes.
CLASSES_LIST =all_classes_names
#CLASSES_LIST = ['2 Tolong', '7 permisi', '1 Maaf']

In [5]:
SEQUENCE_LENGTH = 30#getAverageSequenceLength()
print(SEQUENCE_LENGTH)

30


In [6]:
def getDatafromNpz(file):
    return file['data'],file['label']
dtrain = np.load('feature_train.npz')
dval = np.load('feature_val.npz')
dtest = np.load('feature_test.npz')

features_train,labels_train = getDatafromNpz(dtrain)
features_test,labels_test = getDatafromNpz(dtest)
features_val,labels_val = getDatafromNpz(dval)

from keras.models import load_model
convlstm_model = load_model("./HASILConvLSTM/convlstm_model___Date_Time_2024_01_15__19_48_10___Loss_0.501771092414856___Accuracy_0.9041666388511658.h5")

In [7]:
from sklearn.datasets import make_classification
from sklearn.preprocessing import label_binarize
labels_test = label_binarize(labels_test, classes=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
print(labels_test[:2])

[[0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]]


In [8]:
y_score = convlstm_model.predict(features_test[1:2])
model_evaluation_history = convlstm_model.evaluate(features_test, labels_test)
print(len(features_test[1:3]))

8/8 [==============================] - 1s 105ms/step - loss: 0.5018 - accuracy: 0.9042
2


In [9]:
convlstm_model.summary()
import time
def current_milli_time():
    return round(time.time() * 1000)
def Average(lst): 
    return sum(lst) / len(lst) 

pred_time_list=[]
for i in range(len(features_test)):
    start = current_milli_time() 
    result = convlstm_model.predict(features_test[i:i+1],verbose=0)
    end = current_milli_time()
    pred_time_list.append(end-start)
print(Average(pred_time_list)) 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 30, 78, 78, 8)     3200      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 15, 39, 39, 8)    0         
 )                                                               
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 15, 37, 37, 16)    13888     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 8, 19, 19, 16)    0         
 3D)                                                             
                                                                 
 flatten (Flatten)           (None, 46208)             0         
                                                                 
 dense (Dense)               (None, 10)                4

In [ ]:
def numberToLabel(i):
    if i == 0:
        return "Maaf"
    if i == 1:
        return "Tolong"
    if i == 2:
        return "Permisi"
    if i == 3:
        return "Halo"
    if i == 4:
        return "Mulai"
    if i == 5:
        return "Berhenti"
    if i == 6:
        return "Lanjut"
    if i == 7:
        return "Sakit"
    if i == 8:
        return "Kembali"
    if i == 9:
        return "Awas"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc

def generate_random_color():
    r = random.random()
    g = random.random()
    b = random.random()
    return (r, g, b)

n_classes = len(all_classes_names)
# Plot linewidth.
lw = 2
# random color list
cyclecolor = []
for i in range(n_classes):
    cyclecolor.append(generate_random_color())
print(cyclecolor)
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(labels_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(labels_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
plt.figure(1)
#plt.plot(fpr["micro"], tpr["micro"],
#         label='micro-average (AUC = {0:0.5f})'
#               ''.format(roc_auc["micro"]),
#         color='deeppink', linestyle=':', linewidth=4)

#plt.plot(fpr["macro"], tpr["macro"],
#         label='macro-average (AUC = {0:0.5f})'
#               ''.format(roc_auc["macro"]),
#         color='navy', linestyle=':', linewidth=4)

#colors = cycle(cyclecolor)#['aqua', 'darkorange', 'cornflowerblue', 'brown',generate_random_color()])
colors = cycle([(0.36304485383006224, 0.13127808794416607, 0.46500653405023196),(0.44784037927230047, 0.24361044497306306, 0.9619475862332232),(0.9097541287531854, 0.4153999995918298, 0.6314691892992672),(0.9431365027204199, 0.32218312786294245, 0.35724299603957155),(0.951999732878447, 0.40032164982498486, 0.7596470545715319),(0.498648681286753, 0.4657004573117104, 0.7560587818444884),(0.08526676666572275, 0.4726985147916237, 0.021651492098737624),(0.3437280047335154, 0.34598655530196465, 0.6061628072574675),(0.2275998170118243, 0.7154946792135707, 0.7034996002472135),(0.35019879829306844, 0.06756675128772494, 0.5546605694666252)])
for i, color in zip(range(n_classes), colors):
    print(color,end=",")
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='{0} (AUC = {1:0.5f})'
             ''.format(numberToLabel(i), roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
#plt.title('Some extension of Receiver operating characteristic to multi-class')
#plt.legend(loc="lower right")
fig1 = plt.gcf()
plt.show()
fig1.savefig("ROCAUCZOOM_HASILLRCN3lyr.jpg",dpi=600)

# Zoom in view of the upper left corner.
plt.figure(2)
plt.xlim(0, 0.5)
plt.ylim(0.6, 1)
#plt.plot(fpr["micro"], tpr["micro"],
#         label='micro-average (AUC = {0:0.5f})'
#               ''.format(roc_auc["micro"]),
#         color='deeppink', linestyle=':', linewidth=4)

#plt.plot(fpr["macro"], tpr["macro"],
#         label='macro-average (AUC = {0:0.5f})'
#               ''.format(roc_auc["macro"]),
#         color='navy', linestyle=':', linewidth=4)

#colors = cycle(cyclecolor)
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='{0} (AUC = {1:0.5f})'
             ''.format(numberToLabel(i), roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
#plt.xlabel('False Positive Rate')
#plt.ylabel('True Positive Rate')
#plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right",fontsize="12")
fig2 = plt.gcf()
plt.show()
fig2.savefig("ROCAUC_HASILLRCN3lyr.jpg",dpi=600)